<a href="https://colab.research.google.com/github/yankchina/ColabNotebook/blob/main/LearningPythonAsyncio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Python Asyncio

Python 异步处理是我目前许多项目中最常见的问题，我还是希望能够用 asyncio 而不是 Multithreading。这样我可以用 `async` + `await` 来写更优雅的代码，并且能够带来更灵活的架构。同时我也想通过这样的方式来做具体事情。


## 学习 Lynn Root 的讲座

我还是希望能从一些实际代码入手来进行学习。

于是我找到 Lynn Root 在 2019 Euro Python Conference 上做的 Lecture

- 视频：[Lynn Root - Advanced asyncio: Solving Real-world Production Problems](https://www.youtube.com/watch?v=sW76-pRkZk8&list=LL&index=1&t=1545s&ab_channel=EuroPythonConference)
- 代码：[https://github.com/econchick/mayhem](https://github.com/econchick/mayhem)

我按照她的讲座内容逐个部分做解读。

这个项目的名称叫做 `mayhem` ，其意思是 混乱。这个与异步处理的内涵是相通的。（2021-12-29 理解）



### Part01

我还是用手动的方式，逐行将 代码写入到 Colab Notebook 中

这里用到 `attrs` 这个库

- 官方网址是 [https://www.attrs.org/en/stable/index.html](https://www.attrs.org/en/stable/index.html)
- 一个中文的介绍博客在这里 [Python 使用 attrs 和 cattrs 实现面向对象编程](https://blog.csdn.net/HHG20171226/article/details/103038077)


其官方代码 `part-0` 用 `mayhem_1.py` -> `mayhem_4.py` 最终给出一个正确的写法。我这里用的是 `mayhem_4.py` 的写法。

原来在 Colab 上只有开通了 后台运行这个权限，才能实现这个 event_loop 效果，否则就会出现 event 暂停的情况。这个就比较麻烦了。


In [ ]:
## 预先需要安装一下这个库
# !pip install attrs

import asyncio #<- 异步处理库
import logging #<- 日志库
import random  #<- 随机数库
import string #<- 字符串库
import attr #<- 简化原有的类

## 配置日志
logging.basicConfig(
    level = logging.INFO,
    format='%(asctime)s,%(msecs)d %(levelname)s: %(message)s',
    datefmt='%H:%M:%S',
)

## 定义了一个 PubSubMessage 类
## 这个类就是异步处理的消息类
@attr.s
class PubSubMessage:
  instance_name = attr.ib()
  message_id  = attr.ib(repr=False)
  host_name = attr.ib(repr=False,init=False)

  def __attrs_post_init__(self):
    self.hostname = u"{0}.example.net".format(self.instance_name)


## 给出一个异步处理的函数
## 这个函数将自动进行消息的生成与发布
## Queue 是队列
## n 是需要生成实例的个数
async def publish(queue,n):
  choices = string.ascii_lowercase + string.digits #<- 消息类的名称是 小写 ASCII 码与数字

  for x in range(1,n+1):
    host_id = ''.join(random.choices(choices,k=4)) #<- 自动生成 4 位小写 ASCII 码+数字的随机字符串
    instance_name = u"cattle-{0}".format(host_id) #<- 实例的名称就是 host_id 加一个前缀
    msg = PubSubMessage(message_id=x,instance_name=instance_name)
    await queue.put(msg) #<- 异步操作，将队列放入到系统中
    logging.info(u"Published {0} of {1} Messages".format(x,n)) ## 写入操作日志

  await queue.put(None) # 发布结束

## 继续队列操作，这个显然是重载了 asnycio 的队列操作
async def consume(queue): 
  while True:
    msg = await queue.get()
    if msg is None:
      break

    logging.info(u"Consumed {0}".format(msg))
    await asyncio.sleep(random.random())

## 原来样例代码中 `mayhem_4.py` 的 `main` 函数
def method_4():
  ## 构建一个队列
  queue = asyncio.Queue() 
  ## 构建一个循环
  loop = asyncio.get_event_loop()

  try:
    loop.create_task(publish(queue,10))
    loop.create_task(consume(queue))
    loop.run_forever()
  except KeyboardInterrupt:
    logging.info("Process interrupted")
  finally:
    #loop.run_until_complete(loop.shutdown_asyncgens())
    #loop.close()
    logging.info(u"Successfully shutdown the Mayhem Service")

if __name__ == '__main__':
  method_4()


上面的这段代码在我的本地 Macbook 上是运行成功的，我可以用 `Ctrl-C` 来中断程序的执行，这样我就能够了解到原来的系统是可以执行的。下面进入到 PART2。

我对比了 Lynn Root 给的代码 `part-1/mayhem_1.py` -> `mayhem_9.py`

- 增加了对于队列的控制
- 增加了操作类中的一些信息

没有解决的问题就在其 `consume` 函数，当用户强制中止这个程序，则会出现 Task 错误，其主要原因就是 Loop 已经结束，而 `consume` 仍然在队列中获取信息。

但通过这个程序，我大致能够了解到 `asyncio` 的运行规律了。这其实就是一篇代码模板，我可基于这个模板来做后面的研发工作。

----
@now END（2021-12-29 10:44）